In [1]:
from keras.applications import resnet50

Using TensorFlow backend.


In [2]:
model = resnet50.ResNet50(weights = 'imagenet' , 
                    include_top = False ,
                    input_shape = (224 , 224 , 3) )

c:\users\parihar\anaconda3\envs\mlops\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [3]:
for layer in (model.layers):
    layer.trainable = False 

In [4]:
for layer in (model.layers):
    print(layer.__class__.__name__ , layer.trainable)

InputLayer False
ZeroPadding2D False
Conv2D False
BatchNormalization False
Activation False
ZeroPadding2D False
MaxPooling2D False
Conv2D False
BatchNormalization False
Activation False
Conv2D False
BatchNormalization False
Activation False
Conv2D False
Conv2D False
BatchNormalization False
BatchNormalization False
Add False
Activation False
Conv2D False
BatchNormalization False
Activation False
Conv2D False
BatchNormalization False
Activation False
Conv2D False
BatchNormalization False
Add False
Activation False
Conv2D False
BatchNormalization False
Activation False
Conv2D False
BatchNormalization False
Activation False
Conv2D False
BatchNormalization False
Add False
Activation False
Conv2D False
BatchNormalization False
Activation False
Conv2D False
BatchNormalization False
Activation False
Conv2D False
Conv2D False
BatchNormalization False
BatchNormalization False
Add False
Activation False
Conv2D False
BatchNormalization False
Activation False
Conv2D False
BatchNormalization False


In [5]:
model.output

<tf.Tensor 'activation_49/Relu:0' shape=(None, 7, 7, 2048) dtype=float32>

In [6]:
from keras.models import Sequential

In [7]:
from keras.layers import Dense

In [8]:
from keras.layers import Flatten

In [9]:
patch_model = model.output

In [10]:
patch_model = Flatten()(patch_model)

In [11]:
patch_model = Dense(units=500 , activation='relu')(patch_model)

In [12]:
patch_model = Dense(units=100 , activation='relu')(patch_model)

In [13]:
patch_model = Dense(units=4 , activation='softmax')(patch_model)

In [14]:
from keras.models import Model

In [15]:
model.input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [16]:
patch_model

<tf.Tensor 'dense_3/Softmax:0' shape=(None, 4) dtype=float32>

In [17]:
newmodel = Model(inputs= model.input , outputs =patch_model )

In [18]:
newmodel.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [19]:

from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'faceRecog_ACNN/train_data'
validation_data_dir = 'faceRecog_ACNN/validation_data'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(224, 224),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(224, 224),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 1600 images belonging to 4 classes.
Found 400 images belonging to 4 classes.


In [20]:
from keras.optimizers import Adam
newmodel.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr = 0.01),
              metrics = ['accuracy'])


In [21]:
newmodel.fit_generator(train_generator , validation_data=validation_generator , epochs=1)

Epoch 1/1
100/100 [==============================] - 892s 9s/step - loss: 26.9620 - accuracy: 0.9337 - val_loss: 123.5299 - val_accuracy: 0.2500


In [23]:
newmodel.save('Fam_FACE_RECOG_ResNet50.h5')


In [24]:
from keras.preprocessing import image

In [25]:
utest = image.load_img('udit6.jpeg' , target_size=(224 , 224 , 3) )

In [26]:
u_test_np = image.img_to_array(utest)

In [27]:
u_test_np.shape

(224, 224, 3)

In [ ]:
#hist.history.get('acc')[-1]

In [28]:
import numpy as np 

In [29]:
u_test_np = np.expand_dims(u_test_np , axis = 0)

In [30]:
u_test_np.shape

(1, 224, 224, 3)

In [31]:
pred = newmodel.predict(u_test_np )

In [32]:
from keras.applications.resnet50 import decode_predictions

In [33]:
pred

array([[0., 0., 1., 0.]], dtype=float32)

In [34]:
train_generator.class_indices

{'Drone': 0, 'Rajendra': 1, 'Udit': 2, 'Usha': 3}